In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.linear_model import Ridge as cpu_Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from tqdm.auto import tqdm, trange
from sklearn.base import BaseEstimator, TransformerMixin
from skorch.callbacks import EarlyStopping, LRScheduler
from skorch.dataset import Dataset
from skorch.helper import predefined_split
import joblib
from helpful_functions import InputLogTransformer, OutputLogTransformer, build_neural_network, make_poly_datasets
import time

spectrum = False
noise = 10
if spectrum:
    num_outputs = 25
    identifier = 'spectrum'
    output_list = ['Bin ' + str(i) for i in range(25)] # training outputs
else:
    num_outputs = 3
    identifier = 'threeEns'
    output_list = ['Max Proton Energy', 'Total Proton Energy', 'Avg Proton Energy']
dfA = pd.read_hdf(f'datasets/fuchs_v5_0_seed-2_train_1274091_noise_{noise}_{identifier}_campaign2.h5', key = 'df')
dfB = pd.read_hdf(f'datasets/fuchs_v5_0_seed-2_train_campaign1_pct_100_noise_{noise}.h5', key='df')
#df = pd.concat([dfA, dfB], ignore_index=True).fillna(0)
df = pd.read_hdf(f'datasets/fuchs_v5_0_seed-2_train_1525000_noise_{noise}_threeEns_.h5', key='df').fillna(0)
df.loc[:, output_list] = df.loc[:, output_list].clip(1e-2, None)
test_df = pd.read_hdf(f'datasets/fuchs_v5_0_seed-2_test_1000000_noise_0_{identifier}_.h5', key = 'df').fillna(0)
test_df.loc[:, output_list] = test_df.loc[:, output_list].clip(1e-2, None)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
datype = np.float32
num_inputs = 4
num_outputs = 3
batch_size = 2**14
patience=5

input_list = ['Intensity', 'Target Thickness', 'Focal Distance', 'Contrast'] # independent variables
time_list = []
mape_list = []
point_list = []
uncorrected_list = []
# testing_list = ['Max Exact Energy', 'Total Exact Energy', 'Avg Exact Energy'] # testing set outputs, deprecated

#shuffle dataset to allow for random sampling
df = df.sample(frac=1,random_state=42).reset_index(drop=True)
X = np.array(df[input_list],dtype=datype)
y = np.array(df[output_list],dtype=datype)
#X_train0, y_train0, X_val0, y_val0, input_transformer, output_transformer = make_datasets(X, y, random_state=42)
data_fractions = np.linspace(0, 1, 7)[1:]
num_pts_tot = X.shape[0]

X_test = np.array(test_df[input_list],dtype=datype)
y_test = np.array(test_df[output_list],dtype=datype)

# Polynomial determined by hyperparameter optimization
degree = 7
alpha = 1e-3

for frac in data_fractions:
    np.random.seed(42)
    torch.manual_seed(42)
    t0 = time.time()
    num_pts = int(num_pts_tot * frac + 0.5)
    num_train_pts = int(0.8 * num_pts + 0.5)
    print("training", num_train_pts)
    X_train, y_train, X_val, y_val, input_transformer, output_transformer = make_poly_datasets(X[:num_pts, :], y[:num_pts, :], random_state=42)
    poly = PolynomialFeatures(degree=degree)
    ridge = cpu_Ridge(alpha=alpha)
    model = make_pipeline(poly, ridge)
    model.fit(X_train, y_train)
    y_pred = output_transformer.inverse_transform(model.predict(X_val))
    y_train_pred = output_transformer.inverse_transform(model.predict(X_train))
    y_train_true = output_transformer.inverse_transform(y_train)
    correction_factors = []
    for i in range(num_outputs):
        α = np.mean(y_train_true[:, i]/y_train_pred[:, i])
        correction_factors.append(α)
    y_train_pred_corrected = y_train_pred.copy()
    for i in range(num_outputs):
        y_train_pred_corrected[:, i] *= correction_factors[i]
    X_train_true = input_transformer.inverse_transform(X_train)
    y_val_true = output_transformer.inverse_transform(y_val)
    y_train_pred = output_transformer.inverse_transform(model.predict(X_train))
    X_test_scaled = input_transformer.transform(X_test)
    y_test_pred = output_transformer.inverse_transform(model.predict(X_test_scaled))
    y_test_pred_corrected = y_test_pred.copy()
    for i in range(num_outputs):
        y_test_pred_corrected[:, i] *= correction_factors[i]
    print("Model performance on training data:", mean_absolute_percentage_error(y_train_true, y_train_pred)*100)
    print("Model performance on validation data", mean_absolute_percentage_error(y_val_true, y_pred)*100)
    mape_uncorrected = mean_absolute_percentage_error(y_test, y_test_pred)*100
    print("Model performance on blind randomized testing data:", mape_uncorrected)
    mape = mean_absolute_percentage_error(y_test, y_test_pred_corrected)*100
    print("Corrected model performance on blind randomized testing data:", mape)
    t1 = time.time()
    del_t = t1 - t0
    print("Time", del_t)
    time_list.append(del_t)
    mape_list.append(mape)
    point_list.append(num_train_pts)
    uncorrected_list.append(mape_uncorrected)

df = pd.DataFrame({'points': point_list, 'mape': uncorrected_list, 'corrected': mape_list, 'time': time_list})
df.to_csv('results/Poly_data_split.csv', index=False)

training 203334
Model performance on training data: 1669.4345474243164
Model performance on validation data 1791.1510467529297
Model performance on blind randomized testing data: 305.8645963668823
Corrected model performance on blind randomized testing data: 459.8729610443115
Time 6.201056718826294
training 406666
Model performance on training data: 1370.8828926086426
Model performance on validation data 1528.0974388122559
Model performance on blind randomized testing data: 254.7797679901123
Corrected model performance on blind randomized testing data: 386.35880947113037
Time 11.288595199584961
training 610000
Model performance on training data: 1364.307689666748
Model performance on validation data 1169.0274238586426
Model performance on blind randomized testing data: 236.87222003936768
Corrected model performance on blind randomized testing data: 356.25202655792236
Time 17.344545602798462
training 813334
Model performance on training data: 1262.6300811767578
Model performance on vali

In [2]:
import numpy as np
import pandas as pd
import torch
from sklearn.linear_model import Ridge as cpu_Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from tqdm.auto import tqdm, trange
from sklearn.base import BaseEstimator, TransformerMixin
from skorch.callbacks import EarlyStopping, LRScheduler
from skorch.dataset import Dataset
from skorch.helper import predefined_split
import joblib
from helpful_functions import make_poly_datasets
import time

spectrum = False
noise = 10
if spectrum:
    num_outputs = 25
    identifier = 'spectrum'
    output_list = ['Bin ' + str(i) for i in range(25)] # training outputs
else:
    num_outputs = 3
    identifier = 'threeEns'
    output_list = ['Max Proton Energy', 'Total Proton Energy', 'Avg Proton Energy']

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
datype = np.float32
num_inputs = 4
num_outputs = 3

input_list = ['Intensity', 'Target Thickness', 'Focal Distance', 'Contrast'] # independent variables
time_list = []
mape_list = []
noise_list = []
uncorrected_list = []

noise_range = np.arange(0, 31, 5)


test_df = pd.read_hdf(f'datasets/fuchs_v5_0_seed-2_test_1000000_noise_0_{identifier}_.h5', key = 'df').fillna(0)
test_df.loc[:, output_list] = test_df.loc[:, output_list].clip(1e-2, None)

# Polynomial hyperparameters determined by optimization
degree = 7
alpha = 1e-3

for noise in noise_range:
    np.random.seed(42)
    torch.manual_seed(42)
    t0 = time.time()

    df = pd.read_hdf(f'datasets/fuchs_v5_0_seed-2_train_1525000_noise_{noise}_threeEns_.h5', key='df').fillna(0)
    df.loc[:, output_list] = df.loc[:, output_list].clip(1e-2, None)


    # testing_list = ['Max Exact Energy', 'Total Exact Energy', 'Avg Exact Energy'] # testing set outputs, deprecated

    #shuffle dataset to allow for random sampling
    df = df.sample(frac=1,random_state=42).reset_index(drop=True)
    X = np.array(df[input_list],dtype=datype)
    y = np.array(df[output_list],dtype=datype)
    #X_train0, y_train0, X_val0, y_val0, input_transformer, output_transformer = make_datasets(X, y, random_state=42)

    X_test = np.array(test_df[input_list],dtype=datype)
    y_test = np.array(test_df[output_list],dtype=datype)

    print(f"Training on {noise}% noise")
    X_train, y_train, X_val, y_val, input_transformer, output_transformer = make_poly_datasets(X, y, random_state=42)
    poly = PolynomialFeatures(degree=degree)
    ridge = cpu_Ridge(alpha=alpha)
    model = make_pipeline(poly, ridge)
    model.fit(X_train, y_train)
    y_pred = output_transformer.inverse_transform(model.predict(X_val))
    y_train_pred = output_transformer.inverse_transform(model.predict(X_train))
    y_train_true = output_transformer.inverse_transform(y_train)
    correction_factors = []
    for i in range(num_outputs):
        α = np.mean(y_train_true[:, i]/y_train_pred[:, i])
        correction_factors.append(α)
    y_train_pred_corrected = y_train_pred.copy()
    for i in range(num_outputs):
        y_train_pred_corrected[:, i] *= correction_factors[i]
    X_train_true = input_transformer.inverse_transform(X_train)
    y_val_true = output_transformer.inverse_transform(y_val)
    y_train_pred = output_transformer.inverse_transform(model.predict(X_train))
    X_test_scaled = input_transformer.transform(X_test)
    y_test_pred = output_transformer.inverse_transform(model.predict(X_test_scaled))
    y_test_pred_corrected = y_test_pred.copy()
    for i in range(num_outputs):
        y_test_pred_corrected[:, i] *= correction_factors[i]
    print("Model performance on training data:", mean_absolute_percentage_error(y_train, y_train_pred)*100)
    print("Model performance on validation data", mean_absolute_percentage_error(y_val, y_pred)*100)
    mape_uncorrected = mean_absolute_percentage_error(y_test, y_test_pred)*100
    print("Model performance on blind randomized testing data:", mape_uncorrected)
    mape = mean_absolute_percentage_error(y_test, y_test_pred_corrected)*100
    print("Corrected model performance on blind randomized testing data:", mape)
    t1 = time.time()
    del_t = t1 - t0
    print("Time", del_t)
    time_list.append(del_t)
    mape_list.append(mape)
    noise_list.append(noise)
    uncorrected_list.append(mape_uncorrected)

    df = pd.DataFrame({'noise level': noise_list, 'mape': uncorrected_list, 'corrected': mape_list, 'time': time_list})
    df.to_csv('results/Poly_noise_split.csv', index=False)

Training on 0% noise
Model performance on training data: 4.24430043922432e+16
Model performance on validation data 4.35588771610624e+16
Model performance on blind randomized testing data: 208.89930725097656
Corrected model performance on blind randomized testing data: 309.9773406982422
Time 35.72807478904724
Training on 5% noise
Model performance on training data: 4.24203081744384e+16
Model performance on validation data 4.35017339633664e+16
Model performance on blind randomized testing data: 208.61282348632812
Corrected model performance on blind randomized testing data: 309.9245071411133
Time 35.65686058998108
Training on 10% noise
Model performance on training data: 2.17431159078912e+16
Model performance on validation data 2.08823876845568e+16
Model performance on blind randomized testing data: 208.92634391784668
Corrected model performance on blind randomized testing data: 311.06014251708984
Time 35.55851674079895
Training on 15% noise
Model performance on training data: 4.21266733